In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('datas.txt', header=None, delimiter=r"\s+")

In [ ]:
df.head()

,0,1,2,3,4,5
0,18,19,25,33,42,54
1,24,27,40,41,44,45
2,5,7,17,18,19,56
3,2,6,7,10,31,47
4,9,24,28,38,41,51


In [ ]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data = transformed_dataset, index= df.index)

In [ ]:
number_of_rows = df.values.shape[0]
window_length = 10
number_of_features = df.values.shape[1]

In [ ]:
train = np.empty([number_of_rows-window_length,window_length, number_of_features],dtype=float)
label = np.empty([number_of_rows-window_length, number_of_features], dtype=float)
window_length = 10

for i in range(0,number_of_rows-window_length):
  train[i] = transformed_df.iloc[i:i+window_length,0: number_of_features]
  label[i] = transformed_df.iloc[i+window_length: i+window_length+1, 0:number_of_features]

In [ ]:
train.shape

(446, 10, 6)

In [ ]:
label.shape

(446, 6)

In [ ]:
train[0]

array([[ 1.35433917,  0.17549626, -0.11338222, -0.17281006, -0.21655597,
         0.17733497],
       [ 2.22355615,  1.08102719,  1.40727349,  0.62822057,  0.01689444,
        -1.14590633],
       [-0.52896428, -1.18280013, -0.9243986 , -1.6747425 , -2.90123564,
         0.47138859],
       [-0.96357277, -1.2959915 , -1.93816908, -2.47577313, -1.5005332 ,
        -0.85185271],
       [ 0.05051371,  0.7414531 ,  0.19074892,  0.32783408, -0.33328117,
        -0.26374546],
       [-1.10844226, -0.05088647,  1.20451939,  0.62822057,  0.95069606,
         0.6184154 ],
       [-1.10844226,  0.17549626, -0.51889041,  0.22770525, -0.33328117,
        -0.26374546],
       [ 0.1953832 ,  1.30740993,  1.91415873,  1.4292512 ,  1.41759688,
         0.6184154 ],
       [-0.23922529, -0.05088647, -0.62026746, -0.47319655, -1.03363239,
        -2.0280672 ],
       [ 1.06460018,  0.51507036,  0.59625711, -0.07268123,  0.36707005,
        -0.7048259 ]])

In [ ]:
train[1]

array([[ 2.22355615,  1.08102719,  1.40727349,  0.62822057,  0.01689444,
        -1.14590633],
       [-0.52896428, -1.18280013, -0.9243986 , -1.6747425 , -2.90123564,
         0.47138859],
       [-0.96357277, -1.2959915 , -1.93816908, -2.47577313, -1.5005332 ,
        -0.85185271],
       [ 0.05051371,  0.7414531 ,  0.19074892,  0.32783408, -0.33328117,
        -0.26374546],
       [-1.10844226, -0.05088647,  1.20451939,  0.62822057,  0.95069606,
         0.6184154 ],
       [-1.10844226,  0.17549626, -0.51889041,  0.22770525, -0.33328117,
        -0.26374546],
       [ 0.1953832 ,  1.30740993,  1.91415873,  1.4292512 ,  1.41759688,
         0.6184154 ],
       [-0.23922529, -0.05088647, -0.62026746, -0.47319655, -1.03363239,
        -2.0280672 ],
       [ 1.06460018,  0.51507036,  0.59625711, -0.07268123,  0.36707005,
        -0.7048259 ],
       [-0.67383378, -1.18280013, -1.1271527 ,  0.42796291,  1.41759688,
         1.05949584]])

In [ ]:
label[0]

array([-0.67383378, -1.18280013, -1.1271527 ,  0.42796291,  1.41759688,
        1.05949584])

In [ ]:
label[1]

array([-0.67383378, -0.9564174 ,  0.29212597,  0.82847823,  0.36707005,
        0.03030816])

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import numpy as np

batch_size = 6

In [ ]:
model = Sequential()
model.add(LSTM(56,activation = 'relu',
               input_shape = (window_length,number_of_features),
               return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(56,activation='relu',
               return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(number_of_features))
model.compile(loss='mse',optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
for i in range(10,101,10):
    model.fit(train,label,batch_size = 6, epochs = i)
    to_predict = pd.read_csv('datas.txt', header=None, delimiter=r"\s+")

    scaled_to_predict = scaler.transform(to_predict[:window_length])
    scaled_predicted_output_1 = model.predict(np.array([scaled_to_predict]))
    print(scaler.inverse_transform(scaled_predicted_output_1).astype(int)[0])

    step_5_list = []

    for i in range(5,301,5):
        scaled_to_predict = scaler.transform(to_predict[:i])
        scaled_predicted_output_1 = model.predict(np.array([scaled_to_predict]))
        step_5_list.append(scaler.inverse_transform(scaled_predicted_output_1).astype(int)[0])
    from collections import Counter
    number_counts = Counter()

    for array in step_5_list:
        number_counts.update(array)

    for number in range(1, 61):
        count = number_counts[number]
        print(f"Number: {number}, Count: {count}")

    for i in range(len(step_5_list)):
        print((i*5 + 5), ": ", step_5_list[i])

    scaled_to_predict = scaler.transform(to_predict)
    scaled_predicted_output_1 = model.predict(np.array([scaled_to_predict]))
    print(scaler.inverse_transform(scaled_predicted_output_1).astype(int)[0])

Epoch 1/10
75/75 [==============================] - 4s 11ms/step - loss: 1.0017 - accuracy: 0.1861
Epoch 2/10
75/75 [==============================] - 1s 12ms/step - loss: 0.9993 - accuracy: 0.1726
Epoch 3/10
75/75 [==============================] - 1s 11ms/step - loss: 0.9976 - accuracy: 0.1771
Epoch 4/10
75/75 [==============================] - 1s 12ms/step - loss: 0.9973 - accuracy: 0.1771
Epoch 5/10
75/75 [==============================] - 1s 12ms/step - loss: 0.9969 - accuracy: 0.1973
Epoch 6/10
75/75 [==============================] - 1s 12ms/step - loss: 0.9892 - accuracy: 0.1839
Epoch 7/10
75/75 [==============================] - 1s 16ms/step - loss: 0.9905 - accuracy: 0.1457
Epoch 8/10
75/75 [==============================] - 1s 18ms/step - loss: 0.9854 - accuracy: 0.1726
Epoch 9/10
75/75 [==============================] - 1s 15ms/step - loss: 0.9773 - accuracy: 0.1682
Epoch 10/10
1/1 [==============================] - 1s 535ms/step
[ 8 17 26 36 45 54]
1/1 [===================

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:1052: RuntimeWarning: overflow encountered in multiply
  X *= self.scale_


1/1 [==============================] - 0s 62ms/step
[-9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808]
Epoch 1/30
75/75 [==============================] - 1s 18ms/step - loss: 0.8219 - accuracy: 0.2758
Epoch 2/30
75/75 [==============================] - 1s 18ms/step - loss: 0.7980 - accuracy: 0.2534
Epoch 3/30
75/75 [==============================] - 1s 14ms/step - loss: 0.8195 - accuracy: 0.2892
Epoch 4/30
75/75 [==============================] - 1s 12ms/step - loss: 0.7756 - accuracy: 0.2915
Epoch 5/30
75/75 [==============================] - 1s 12ms/step - loss: 0.7606 - accuracy: 0.2982
Epoch 6/30
75/75 [==============================] - 1s 12ms/step - loss: 0.7588 - accuracy: 0.2578
Epoch 7/30
75/75 [==============================] - 1s 12ms/step - loss: 0.7841 - accuracy: 0.2489
Epoch 8/30
75/75 [==============================] - 1s 12ms/step - loss: 0.7275 - accuracy: 0.2915
Epoch 9/30
75/75 [=========

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:1052: RuntimeWarning: overflow encountered in multiply
  X *= self.scale_


1/1 [==============================] - 0s 53ms/step
Number: 1, Count: 0
Number: 2, Count: 0
Number: 3, Count: 0
Number: 4, Count: 4
Number: 5, Count: 0
Number: 6, Count: 5
Number: 7, Count: 1
Number: 8, Count: 0
Number: 9, Count: 0
Number: 10, Count: 3
Number: 11, Count: 2
Number: 12, Count: 0
Number: 13, Count: 0
Number: 14, Count: 1
Number: 15, Count: 1
Number: 16, Count: 0
Number: 17, Count: 2
Number: 18, Count: 0
Number: 19, Count: 0
Number: 20, Count: 1
Number: 21, Count: 0
Number: 22, Count: 1
Number: 23, Count: 1
Number: 24, Count: 2
Number: 25, Count: 0
Number: 26, Count: 0
Number: 27, Count: 0
Number: 28, Count: 0
Number: 29, Count: 1
Number: 30, Count: 1
Number: 31, Count: 0
Number: 32, Count: 1
Number: 33, Count: 0
Number: 34, Count: 0
Number: 35, Count: 0
Number: 36, Count: 1
Number: 37, Count: 2
Number: 38, Count: 0
Number: 39, Count: 1
Number: 40, Count: 1
Number: 41, Count: 0
Number: 42, Count: 1
Number: 43, Count: 0
Number: 44, Count: 1
Number: 45, Count: 0
Number: 46, 